In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
TRAIN_CSV = '../input/facial-expression-recognitionferchallenge/fer2013/fer2013/fer2013.csv'

In [ ]:
import sys, os
import pandas as pd
import numpy as np
from keras.applications.resnet50 import ResNet50
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,AveragePooling2D
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import np_utils
from keras.utils import to_categorical

In [ ]:
df = pd.read_csv(TRAIN_CSV)

In [ ]:
df.head()

In [ ]:
emotion_label_to_text = {0:'anger', 1:'disgust', 2:'fear', 3:'happiness', 4: 'sadness', 5: 'surprise', 6: 'neutral'}

In [ ]:
X_train,train_y,X_test,test_y=[],[],[],[]

In [ ]:
for index, row in df.iterrows():
    val=row['pixels'].split(" ")
    try:
        if 'Training' in row['Usage']:
           X_train.append(np.array(val,'float32'))
           train_y.append(row['emotion'])
        elif 'PublicTest' in row['Usage']:
           X_test.append(np.array(val,'float32'))
           test_y.append(row['emotion'])
    except:
        print(f"error occured at index :{index} and row:{row}")

In [ ]:
X_train[:5]

In [ ]:
X_train = np.array(X_train)
X_test = np.array(X_test)
train_y = np.array(train_y)
test_y = np.array(test_y)

In [ ]:
X_train -= np.mean(X_train, axis=0)
X_train /= np.std(X_train, axis=0)

X_test -= np.mean(X_test, axis=0)
X_test /= np.std(X_test, axis=0)

X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)

X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(X_train[0].squeeze())
X_train.shape

In [ ]:
model = Sequential()
model.add(Conv2D(64, (3,3), padding='same',activation='relu', input_shape=(48, 48, 1)))
model.add(MaxPooling2D(2,2))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Conv2D(128, (3,3), padding='same',activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Conv2D(256, (3,3), padding='same',activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(BatchNormalization())
model.add(Conv2D(256, (3,3), padding='same',activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Conv2D(512, (3,3), padding='same',activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dense(7, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
from keras.optimizers import Adam

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0002), loss = 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train, to_categorical(train_y), batch_size=64, epochs=40, validation_data =(X_test, to_categorical(test_y)))

In [ ]:
from keras.models import load_model

model.save('facefeatures_new_model.h5')

In [ ]:
fer_json = model.to_json()
with open("fer.json", "w") as json_file:
    json_file.write(fer_json)